# ハイパーパラメータ 実践編
入門編では、簡単な関数の最適解を求める問題を通してOptunaの基本的な使い方を学習しました。

実践編では、いよいよ機械学習のハイパーパラメータ最適化に、Optunaを活用してみましょう。

最初に次のプログラムを実行して、必要なファイルのダウンロードと、Optunaのインストールを行ってください。

In [ ]:
# 前準備
!curl -O https://raw.githubusercontent.com/tendo-sms/python_intermediate/refs/heads/main/2024/kadai1/CrabAgePrediction.csv
%pip install optuna

# 事前準備 (乱数seedの固定)
入門編と同様に、乱数seedを固定して実行結果が毎回同じになるようにしておきます。

次のコードを実行してください。

In [ ]:
import numpy as np
import random

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 100

np.random.seed(SEED)
random.seed(SEED)

もし、これ以降のプログラムを実行したときに「変数SEEDが存在しない」というエラーとなった場合は、上記のプログラムが実行されていないことが原因です。忘れずに実行してください。

# 機械学習におけるハイパーパラメータの最適化
今回は、機械学習のグループワークでご紹介したKaggleで公開されている、カニ🦀の年齢データを用いたサンプルコードをご紹介します。

## 注意事項
本資料中のサンプルプログラムは講義用のため、時間がかかりすぎないようにデータ数や試行回数を小さくしています。

講義用に分かりやすい結果が出るような乱数seedに固定してしていますが、乱数seedを変更したり、固定せずに実行したりすると、良い結果が出ないことがあります。

講義時間内でOptunaの使い方を覚えていただくという目的にフォーカスしているため、ご了承いただければ幸いです。

## データの内容確認
最初の操作で「CrabAgePrediction.csv」が皆さんの環境にダウンロードされています。

ひとまず、データの中身をデータフレームにして覗いてみましょう。

次のコードを実行してください。

In [ ]:
import pandas as pd

#データセットの読み込み
df = pd.read_csv("CrabAgePrediction.csv")
df

次の特徴量があることが分かります。

- Sex：性別
- Length：長さ
- Diameter：直径
- Height：高さ
- Weight：重量
- Shucked Weight：殻付き重量
- Viscera Weight：内臓重量
- Shell Weight：殻重量
- Age：年齢

今回は、回帰モデルを用いてカニの身体的特徴から年齢を予測します。

## データの準備
機械学習に用いるため、データの加工などを行います。

### 説明変数・目的変数の選択
データを見ると、性別を示す値がF(メス)、M(オス)、I(不明)というように、名義尺度の質的変数になっています。

質的変数は回帰モデルの説明変数としては使いづらいので、今回は性別を使用しないことにします。(数字にマッピングして使用することもできますが、ノイズとなってしまうことが多いです。)

また、今回は年齢予測ということで、目的変数は年齢となります。

次のコードを実行してください。

In [ ]:
# 説明変数と目的変数
X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
y = df.iloc[0:, 8]   # 目的変数 (年齢)

### データの標準化
回帰モデルはデータのスケールに敏感であるため、データの標準化を行います。

次のコードを実行してください。

In [ ]:
from sklearn.preprocessing import StandardScaler

# データの標準化
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

### データセットの分割と件数の絞り込み
`train_test_split`を使って、データセットを訓練データとテストデータに分割します。

元のデータは3,893件ありますが、すべてを対象としてしまうと学習や評価に時間がかかるため、今回は講義用に訓練データ400件、テストデータ100件に絞り込みます。

`train_test_split`は、引数`train_size`および`test_size`を指定することで、データをランダムにサンプリングできます。

次のコードを実行してください。

In [ ]:
from sklearn.model_selection import train_test_split

SIZE_TRAIN = 400
SIZE_TEST = 100

# ランダムにデータをサンプリングして分割 (乱数seedを固定)
X_train, X_test, y_train, y_test = train_test_split(X_std, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST, random_state=SEED)

データが絞り込まれていることを確認してみましょう。

次のコードを実行してください。

In [ ]:
# len関数で件数をカウント
print(f"X_train: {len(X_train)}")
print(f"y_train: {len(y_train)}")
print(f"X_test:  {len(X_test)}")
print(f"y_test:  {len(y_test)}")

## ハイパーパラメータ最適化前の実行結果
今回は、ランダムフォレストの回帰モデルであるRandomForestRegressorを使用します。

まずは、デフォルトのハイパーパラメータで機械学習を行ってみます。

学習できたら、テストデータから4件を抜粋して予測してみましょう。

次のコードを実行してください。

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 学習モデルの定義
model = RandomForestRegressor(
    random_state=SEED  # 乱数を固定するためにSEEDを指定
    )

# 学習
model.fit(X_train, y_train)

# 予測ができているか確認
print(f"Predict: {np.round(model.predict(X_test[:4]), 3)}")
print(f"Correct: {list(y_test[:4])}")

大きく外していることはなさそうです。

### モデルの評価方法
今回は、学習済みモデルの評価方法として決定係数($R^2$スコア)を使用します。

テストデータを使って、決定係数を求めてみます。

次のコードを実行してください。

In [ ]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R2 Score: {r2}")

すべてのテストデータで決定係数を求めると、さほど精度が高くありませんでした。

この決定係数を、ハイパーパラメータの最適化により向上させることを目指します。

### [ご参考] 全体を1つにまとめたプログラム
ここまでで、ハイパーパラメータ最適化前のプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

このコードでは、乱数seedを固定していません。

また、データの準備を`load_data`関数としてまとめています。

~~~python
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def load_data():
    # データセットの読み込み
    df = pd.read_csv("CrabAgePrediction.csv")

    # 説明変数と目的変数
    X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
    y = df.iloc[0:, 8]   # 目的変数 (年齢)

    # データの標準化
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)

    SIZE_TRAIN = 400
    SIZE_TEST = 100

    # ランダムにデータをサンプリングして分割
    return train_test_split(X_std, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST)

# データの読み込み
X_train, X_test, y_train, y_test = load_data()

# 学習モデルの定義
model = RandomForestRegressor()

# 学習
model.fit(X_train, y_train)

# 予測ができているか確認
print(f"Predict: {np.round(model.predict(X_test[:4]), 3)}")
print(f"Correct: {list(y_test[:4])}")

# モデルの評価
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R2 Score: {r2}")
~~~

## ハイパーパラメータ最適化のプログラム作成
ここからいよいよ、ハイパーパラメータの最適化にチャレンジしていきます。

### RandomForestRegressorのハイパーパラメータ
PowerPointの資料でご紹介したとおり、RandomForestRegressorの代表的なハイパーパラメータとしては以下などがあります。

- n_estimators：決定木の数
- max_depth：決定木の最大の深さ
- max_features：分岐を決定するのに用いる説明変数の最大数

今回は上記に加えて、以下のハイパーパラメータも最適化の対象とします。

- criterion：ノードをどのように分割するかを決定する基準

それでは、Optunaを使って最適化してみましょう。

### 目的関数の作成
目的関数を設定します。

次のコードを実行してください。

In [ ]:
import optuna

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 50),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"]),
        random_state=SEED
    )

    # 学習
    model.fit(X_train, y_train)

    # テストデータによる予測
    y_pred = model.predict(X_test)

    # 決定係数を戻り値とする
    return r2_score(y_test, y_pred)

サジェストAPIの`suggest_int`と`suggest_categorical`については、後ほど詳しくご紹介します。

### スタディの作成および実行 ～ 結果の出力
ここからは、入門編でご紹介した使い方とまったく同じです。

次のとおり、スタディの作成、実行、結果の出力を行います。

今回の試行回数は、50回としました。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

最適化前の決定係数 0.4653・・・ と比較して、若干ではありますが決定係数が向上しました。

デフォルト値がそれなりに優秀であること、データ数や試行回数を少なくてしていることなどから、残念ながら大きな向上とはなりませんでした。

ちなみに、試行回数を200回として実行すると以下の結果となり、さらに若干ですが決定係数を向上させることができました。

~~~
Best objective value: 0.48539721742500175
Best parameter: {'n_estimators': 119, 'max_depth': 26, 'max_features': 'log2', 'criterion': 'absolute_error'}
~~~

ハイパーパラメータのブラックボックス最適化も、入門編でご紹介した数式の最適化と全く同じやり方で、簡単に実践できることが体感できたかと思います。

### [ご参考] 全体を1つにまとめたプログラム
ここまでで、Optunaによるハイパーパラメータ最適化のプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

このコードでは、乱数seedを固定していません。そのため、`create_study`関数で`sampler`を指定する必要はありません。

また、データの準備を`load_data`関数としてまとめています。

~~~python
import pandas as pd
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

SIZE_TRAIN = 400
SIZE_TEST = 100

def load_data():
    # データセットの読み込み
    df = pd.read_csv("CrabAgePrediction.csv")

    # 説明変数と目的変数
    X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
    y = df.iloc[0:, 8]   # 目的変数 (年齢)

    # データの標準化
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)

    # ランダムにデータをサンプリングして分割
    return train_test_split(X_std, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST)

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 50),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"])
    )

    # 学習
    model.fit(X_train, y_train)

    # テストデータによる予測
    y_pred = model.predict(X_test)

    # 決定係数を戻り値とする
    return r2_score(y_test, y_pred)

# データの読み込み
X_train, X_test, y_train, y_test = load_data()

# スタディの作成
study = optuna.create_study(
    direction="maximize"
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
~~~

## 結果の可視化と分析
Optunaでは、スタディの結果を可視化・分析するための様々な機能が提供されています。

これらの機能でパラメータと評価値の関係などを分析することで、さらに効率の良い探索を行う方法を検討できます。

ここでは、いくつかの機能をご紹介します。

### データフレームの出力
Studyオブジェクトの`trials_dataframe`メソッドで、各トライアルの結果をPandasのデータフレームとして出力できます。

次のコードを実行してください。

In [ ]:
df_trial = study.trials_dataframe()
df_trial


データフレームは1行が1回のトライアルとなっており、トライアルに関する次のような情報が格納されます。

- 通し番号
- 評価値
- 開始・終了時刻・経過時間
- 試行したパラメータ値
- ステータス

### ハイパーパラメータ重要度の出力
Visualizationオブジェクトの`plot_param_importances`メソッドで、それぞれのハイパーパラメータがどのくらい評価結果に影響したかが分かります。

この影響度は「**ハイパーパラメータ重要度**」と呼ばれます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_param_importances(
    study=study,
    params=["n_estimators", "max_depth", "max_features", "criterion"]
).show()

この結果から、今回のチューニングでは`max_depth`の影響が比較的大きかった一方、`n_estimators`や`max_features`の影響度はかなり小さかったことが分かります。

影響度が非常に小さいパラメータは最適化の対象から外す、などの対応が考えられます。

### パラメータ値と評価値の関係を出力
Visualizationオブジェクトの`plot_slice`メソッドで、それぞれのハイパーパラメータと評価値の関係を散布図により詳細に可視化することができます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_slice(
    study=study,
    params=["n_estimators", "max_depth", "max_features", "criterion"]
).show()

この機能は、パラメータ値の候補を絞り込むときに大変有効です。

たとえば、値がある範囲を外れたら評価値がほとんど同じになる、などの傾向があれば、その部分は候補から外すことで無駄な探索を減らすことができます。

また、パラメータ値と評価値に明確な相関があれば、評価値が低くなる方向の候補は削り、高くなる方向の候補はもっと増やすなどすることができます。

### 等高線プロットの出力
Visualizationオブジェクトの`plot_contour`メソッドで、2つのハイパーパラメータの組み合わせと評価値の関係を、等高線図により詳細に可視化することができます。

例えば、`n_estimators`と`max_depth`の値と、評価値との関係について、等高線図を作成してみます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_contour(
    study=study,
    params=["n_estimators", "max_depth"]
).show()

また、`max_features`も加えた3つのパラメータをプロットすると、組み合わせのパターンごとに等高線図が作成されます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_contour(
    study=study,
    params=["n_estimators", "max_depth", "max_features"]
).show()

パラメータの組み合わせと評価値の関係を分析したいときに、活用したい機能です。

### 探索履歴の出力
ハイパーパラメータ入門編でもご紹介した機能ですが、Visualizationオブジェクトの`plot_optimization_history`メソッドで、各トライアルでの評価値と、その時点でもっとも良い評価値の履歴を出力することができます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

ベイズ最適化であれば、トライアルが進むごとに評価値の高い試行が増えていく傾向が見えれば、効率的な探索ができていると言えそうです。

## サジェストAPI
OptunaのサジェストAPIには、以下のような種類があります。

- `suggest_int`
- `suggest_float`
- `suggest_categorical`

入門編では`suggest_float`を使用しましたが、今回は次のとおり、`suggest_int`と`suggest_categorical`を使用しました。

~~~
n_estimators=trial.suggest_int("n_estimators", 50, 500)
max_depth=trial.suggest_int("max_depth", 3, 50)
max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"])
criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"])
~~~

それぞれのサジェストAPIについて、詳しくご紹介します。

### suggest_int、suggest_float
パラメータの候補が整数型のときは`suggest_int`、浮動小数点数型のときは`suggest_float`を、それぞれ使用します。

文法は共通で、次のとおりです。

~~~python
suggest_int(name, low, high, *[, step, log])
suggest_float(name, low, high, *[, step, log])
~~~
- `name`は、パラメータに付ける名前です。ログ出力や可視化においてラベルとして用いられます。
- `low`、`high`は、パラメータ値の候補の最小値、最大値です。
- `step`が指定された場合、`step`で区切られた離散的なパラメータ値が決定されます。
- `log`が指定された場合、対数スケールでパラメータ値が決定されます。

入門編では、値の範囲が$(-4.5 < x < 4.5, -4.5 < y < 4.5)$であることを示すため、`suggest_float`を用いました。

今回は、`n_estimators`と`max_depth`の候補値が整数型でしたので、`suggest_int`を用いました。

### suggest_categorical
パラメータの候補が文字列ラベルの一覧といった名義尺度であるときは、`suggest_categorical`を使用します。

文法は次のとおりです。

~~~python
suggest_categorical(name, choices)
~~~
- `name`は、パラメータに付ける名前です。
- `choices`は、パラメータ値の候補となるリストです。

今回は、`max_features`と`criterion`の候補値が文字列ラベルから選択するものであったため、`suggest_categorical`を用いました。

# 事前知識の活用
ベイズ最適化において、最初の方のトライアルではパラメータ値の候補を選ぶ基準がないため、ほぼランダムでパラメータ値の候補が決定します。

そのため、的外れなパラメータも含めた大量のトライアルを重ねることで、しだいに効率的な探索が行われていきます。

しかし、もし「あるていど良い結果が出るパラメータ値」が事前に分かっていれば、探索のスタートダッシュを有利にすることができます。

## 探索の初期値設定
最初にOptunaによる最適化を行った例で、200回の試行を行うことで以下の解が得られました。

~~~
Best objective value: 0.48539721742500175
Best parameter: {'n_estimators': 119, 'max_depth': 26, 'max_features': 'log2', 'criterion': 'absolute_error'}
~~~

少なくともこのパラメータは良い結果が出ることが分かっているので、事前知識として活用できそうです。

また、パラメータのデフォルト値は平均的によい結果となる値が採用されているものですので、これも事前知識として活用しましょう。

`max_depth`と`max_features`のデフォルト値はちょっと特殊なので省略しますが、`n_estimators`のデフォルト値は100、`criterion`のデフォルト値は"squared_error"です。

Studyオブジェクトの`enqueue_trial`メソッドを使って、これらの値から探索を開始するよう指示することができます。

~~~python
study.enqueue_trial({"n_estimators": 119, "max_depth": 26, "max_features": "log2", "criterion": "absolute_error"})
study.enqueue_trial({"n_estimators": 100, "criterion": "squared_error"})
~~~
この処理を追加した、以下のプログラムを実行してみましょう。

今回はスタートダッシュの変化を確認したいだけなので、試行回数は10回にとどめています。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
    )

# スタディの実行
study.enqueue_trial({"n_estimators": 292, "max_depth": 24, "max_features": "sqrt", "criterion": "absolute_error"})
study.enqueue_trial({"n_estimators": 100, "criterion": "squared_error"})
study.optimize(objective, n_trials=10)  # 試行回数は10回とする

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

ログの最初の2行を見てみてください。

`enqueue_trial`メソッドで指定したパラメータの探索を最初に行っていることが確認できます。

## 評価済みトライアルの追加
先ほどは、パラメータの値のみを事前知識として利用しました。

それだけでなく、そのパラメータを使用したときの評価値まで判明している場合は、評価値も含めて事前知識として活用できます。

これを行うには、Studyオブジェクトの`add_trial`メソッドを使用します。

~~~python
# 探索空間の設定が必要
search_space = {
    "n_estimators": optuna.distributions.IntDistribution(50, 500),
    "max_depth": optuna.distributions.IntDistribution(3, 50)
}

# トライアルの追加
study.add_trial(optuna.trial.create_trial(
    params={"n_estimators": 292, "max_depth": 24},
    distributions=search_space,
    value=0.485
))
~~~

今回のように講義用の小規模なプログラムではあまり影響が出ないため、実行可能なサンプルコードは省略します。

より効率的な探索を追求する場合は、活用してみてください。

# 多目的最適化
これまでご紹介した例は、評価値が1つでした。しかし現実的には、評価値が2つ以上になるような事例も考えられます。

身近な例で例えると、スマートフォンを購入しようと思ったとき、「価格」「性能」という2つを評価値とすることが考えられます。

これらはどちらか一方が優れているだけではダメで、「あちら立てればこちらが立たぬ」というトレードオフの関係となっています。

機械学習のハイパーパラメータ最適化においては、例えば「精度」に加えて、学習や評価にかかる「時間」の2つを評価値とするパターンが考えられます。

これも一般的には「あちら立てればこちらが立たぬ」のパターンです。

このように、複数の評価値を最大化・最小化するための最適化を、多目的最適化と呼びます。

ここでは、最初の決定係数による評価を行う最適化プログラムをベースに、「評価値の精度」と「最適化の時間」の2つを最適化する例をご紹介します。

## 目的関数の作成
目的関数を作成します。

次のコードを実行してください。

In [ ]:
import time

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators = trial.suggest_int("n_estimators", 10, 500),
        max_depth = trial.suggest_int("max_depth", 2, 30),
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"]),
        random_state=SEED
    )

    start_time = time.time() # 開始時間

    # 学習
    model.fit(X_train, y_train)

    # テストデータによる予測
    y_pred = model.predict(X_test)

    end_time = time.time()   # 終了時間

    # 決定係数と経過時間を戻り値とする
    return r2_score(y_test, y_pred), (end_time - start_time)

モデルの定義は先ほどと同じですが、次の点が異なります。

- 最適化にかかる時間として、学習と評価の合計時間を測定
- 戻り値に評価値と測定時間のタプルを指定

## スタディの作成
スタディを作成します。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    directions=["maximize", "minimize"],
    sampler=optuna.samplers.NSGAIISampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )

作成にはこれまでのとおり`create_study`を使用しますが、引数が異なります。

まず、これまでの`direction`に代わり、`directions`を指定しています。

~~~python
directions=["maximize", "minimize"],
~~~

`directions`の引数はリストであり、1つ目の評価値(精度)は最大に、2つ目の評価値(時間)は最小となるよう、`["maximize", "minimize"]`としています。

また、`sampler`にはこれまでのTPESamplerではなく、NSGAIISamplerを指定しています。

~~~python
sampler=optuna.samplers.NSGAIISampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
~~~

NSGAIISamplerは、多目的最適化においてTPESamplerよりも高いパフォーマンスを発揮します。

NSGAIISamplerはベイズ最適化ではなく、進化計算と呼ばれるアルゴリズムをベースとしたサンプラーです。

なお`sampler`の指定を省略した場合、単目的最適化であればTPESamplerを、多目的最適化であればNSGAIISamplerが、自動的にデフォルトとして使用されます。

## [ご参考] Optunaが提供するサンプラー
Optunaは、次のようなサンプラーを提供しています。

- BaseSampler
- GridSampler
- RandomSampler
- TPESampler (単目的最適化のデフォルト)
- CmaEsSampler
- GPSampler
- PartialFixedSampler
- NSGAIISampler (多目的最適化のデフォルト)
- NSGAIIISampler
- QMCSampler
- BruteForceSampler

それぞれの特徴が公式リファレンスにまとめられていますので、興味のある方は見てみてください。
([optuna.samplers](https://optuna.readthedocs.io/en/stable/reference/samplers/))

また、Optuna向け機能共有プラットフォーム「OptunaHub」にて、AutoSamplerという機能が公開されています。

これを利用すると、試行回数、パラメータ数、探索空間などから最適と見込まれるサンプラーが自動的に選択されます。(optunahubパッケージのインストールが必要です。)

~~~
study = optuna.create_study(
    sampler=optunahub.load_module(
        "samplers/auto_sampler"
    ).AutoSampler()  # Automatically selects an algorithm internally
)
~~~
参考記事：[AutoSampler: Automatic Selection of Optimization Algorithms in Optuna](https://medium.com/optuna/autosampler-automatic-selection-of-optimization-algorithms-in-optuna-1443875fd8f9)

## スタディの実行と結果の出力
スタディを実行し、結果を出力します。

次のコードを実行してください。

In [ ]:
# スタディを作成
# (このコードブロックだけを繰り返し実行してもスタディが最初からとなるよう、スタディを作り直す)
study = optuna.create_study(
    directions=["maximize", "minimize"],
    sampler=optuna.samplers.NSGAIISampler(seed=SEED)
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")

スタディの実行はこれまでと同じです。試行回数は50としています。

~~~python
# スタディの実行
study.optimize(objective, n_trials=50)
~~~

今回のような多目的最適化では、前述のとおり評価結果が「あちら立てればこちらが立たぬ」の状態です。

そのため、最適なパラメータの組み合わせを一つに確定することはできません。

スタディの結果は、`best_trials`という変数にリストとして格納されています。これを最適化の結果として出力します。

~~~python
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")
~~~

実行結果を見ると、複数の解が出力されていることが分かります。

このリストに含まれる結果は「**パレートフロント**」と呼ばれており、「他のすべてのトライアルに対して、いずれかの評価値が優れている」結果の集合です。


### [参考] パレートフロントのイメージ
パレートフロントのイメージを図示します。このイメージでは、評価1、評価2の両方とも、値が小さい方が優れているとします。

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2024/images/pareto_front.png" width="35%" alt="パレートフロントのイメージ">

この例でのパレートフロントは、①と②になります。

- ①は、評価1において他の全てのトライアルに勝っています。
- ②は、評価2において他の全てのトライアルに勝っています。
- ③と④は、評価1と評価2のどちらも、他のいずれかのトライアルに劣っています。

## [ご参考] 全体を1つにまとめたプログラム
ここまでで、Optunaによる多目的最適化のプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

このコードでは、乱数seedを固定していません。そのため、`create_study`関数で`sampler`を指定する必要はありません。

また、データの準備を`load_data`関数としてまとめています。

~~~python
import time
import pandas as pd
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

SIZE_TRAIN = 400
SIZE_TEST = 100

def load_data():
    # データセットの読み込み
    df = pd.read_csv("CrabAgePrediction.csv")

    # 説明変数と目的変数
    X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
    y = df.iloc[0:, 8]   # 目的変数 (年齢)

    # データの標準化
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)

    # ランダムにデータをサンプリングして分割 (乱数seedを固定)
    return train_test_split(X_std, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST)

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators = trial.suggest_int("n_estimators", 10, 500),
        max_depth = trial.suggest_int("max_depth", 2, 30),
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"])
    )

    start_time = time.time() # 開始時間

    # 学習
    model.fit(X_train, y_train)

    # テストデータによる予測
    y_pred = model.predict(X_test)

    end_time = time.time()   # 終了時間

    # 決定係数を戻り値とする
    return r2_score(y_test, y_pred), (end_time - start_time)

# データの読み込み
X_train, X_test, y_train, y_test = load_data()

# スタディの作成
study = optuna.create_study(
    directions=["maximize", "minimize"],
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")
~~~

## 評価結果の可視化
結果のリストをただ並べても、その中でどれを選ぶべきか、ぱっと見では判断できないでしょう。

そこで、評価結果を可視化してみます。Visualizationオブジェクトの`plot_pareto_front`メソッドで、トライアルの結果とパレートフロントを可視化できます。

次のコードを実行してください。

In [ ]:
# すべてのトライアルをプロット
optuna.visualization.plot_pareto_front(
	   study,
	   target_names=["R2", "Time"]
	   ).show()

横軸を精度、縦軸を時間として、各トライアルの結果がプロットされています。

精度が良くなるほど速度は遅くなる傾向が分かります。

赤いカラーバーで示される点が`best_trials`(パレートフロント)、青いカラーバーで示される点がその他のトライアル結果です。

また次のとおり、`include_dominated_trials`引数にFalseを指定して`plot_pareto_front`メソッドを実行すると、パレートフロントのみプロットすることができます。

次のコードを実行してください。

In [ ]:
# Study.best_trials だけをプロット
optuna.visualization.plot_pareto_front(
	   study,
	   target_names=["R2", "Time"],
	   include_dominated_trials=False
	   ).show()

パレートフロントのみが表示されました。

多目的最適化では、パレートフロントの結果の中から、用途や目的に合わせてどの結果を採用するのか決めるのがよいでしょう。

# 交差検証(クロスバリデーション)
これまで、データセットを訓練データとテストデータに分け、テストデータでモデルを評価する方法をご紹介してきました。

ここでは、もっと簡単に、かつ精度の高い評価を行う手法として、**交差検証(クロスバリデーション)**をご紹介します。

交差検証はブラックボックス最適化の固有な機能というわけではありませんが、機械学習の分野において非常に便利な機能ですので、実例を交えてご紹介します。

## データの準備
交差検証では、データセットを訓練データとテストデータに分割する必要がありません。

今回は、300件のデータをランダムに取り出して使用します。

train_test_splitを応用して取り出すことができます。

次のコードを実行してください。

In [ ]:
SIZE_DATA = 300

df = pd.read_csv("CrabAgePrediction.csv")

# 説明変数と目的変数
X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
y = df.iloc[0:, 8]   # 目的変数

# データの標準化
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# ランダムにデータを300件サンプリング
X_cv, _, y_cv, _ = train_test_split(X_std, y, train_size=SIZE_DATA, random_state=SEED)

`train_size`引数で、訓練データを300件取り出し、テストデータは捨ててしまいます。

こうして得られた訓練データを、訓練データとしてではなく、交差検証用のデータとして使用します。

## 目的関数の作成
目的関数を作成します。

次のコードを実行してください。

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators = trial.suggest_int("n_estimators", 50, 500),
        max_depth = trial.suggest_int("max_depth", 3, 50),
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"]),
        random_state=SEED
    )

    # 交差検証によるモデルの評価
    score = cross_val_score(model, X_cv, y_cv, cv=3)
    return score.mean()

モデルの定義は先ほどと同じですが、モデルの評価の部分が異なっています。

交差検証を用いる場合、データをテストデータと訓練データに分割したり、コード中で`model.fit()`を行う必要もありません。

`scikit-learn`の機能である`cross_val_score`関数が、以下のことを自動的に行っています。

1. データを3個に分割します。分割数は引数`cv`で指定しています。
2. 分割した1個めをテストデータ、2個目と3個目を訓練データとして、学習(`model.fit()`)と評価を行います。
3. 分割した2個めをテストデータ、1個目と3個目を訓練データとして、学習と評価を行います。
4. 分割した3個めをテストデータ、1個目と2個目を訓練データとして、学習と評価を行います。
5. 上記2.～4.の評価結果を、リストとして返します。

なお評価値は、採用するモデルにより自動的に決定されます(引数で明示指定もできます)。

例えば、回帰系であれば決定係数$R^2$、分類系であれば正解率(Accuracy)となります。

今回は回帰系のRandomForestRegressorなので、決定係数$R^2$が評価値となります。

最終的に、すべての評価における決定係数の平均を評価値としています。

## スタディの作成および実行 ～ 結果の出力
スタディの作成、実行、結果の出力はこれまでと同じです。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

交差検証は、評価に時間がかかるのが難点です。

講義用にデータ数を少なめとしたことに加え、交差検証はより厳しいチェックとなりますので、最終的な評価値は低下しました。

とはいえコード自体は非常に簡単ですので、より精度の高い評価を行いたいときは、交差検証をぜひ利用してみてください。

## [ご参考] 全体を1つにまとめたプログラム
ここまでで、交差検証を利用したプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

このコードでは、乱数seedを固定していません。そのため、`create_study`関数で`sampler`を指定する必要はありません。

また、データの準備を`load_data`関数としてまとめています。

~~~python
import time
import pandas as pd
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def load_data():
    # データセットの読み込み
    df = pd.read_csv("CrabAgePrediction.csv")

    # 説明変数と目的変数
    X = df.iloc[0:, 1:8] # 説明変数 (0列目の性別は除外)
    y = df.iloc[0:, 8]   # 目的変数 (年齢)

    # データの標準化
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)

    SIZE_TRAIN = 400
    SIZE_TEST = 100

    # ランダムにデータをサンプリングして分割 (乱数seedを固定)
    return train_test_split(X_std, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST)

def objective(trial):
    # ランダムフォレスト回帰モデルの定義
    model = RandomForestRegressor(
        n_estimators = trial.suggest_int("n_estimators", 10, 500),
        max_depth = trial.suggest_int("max_depth", 2, 30),
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        criterion=trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"])
    )

    start_time = time.time() # 開始時間

    # 学習
    model.fit(X_train, y_train)

    # テストデータによる予測
    y_pred = model.predict(X_test)

    end_time = time.time()   # 終了時間

    # 決定係数を戻り値とする
    return r2_score(y_test, y_pred), (end_time - start_time)

# データの読み込み
X_train, X_test, y_train, y_test = load_data()

# スタディの作成
study = optuna.create_study(
    directions=["maximize", "minimize"],
    )

# スタディの実行
study.optimize(objective, n_trials=50)

# 結果の出力
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")
~~~

以上、Optunaによりハイパーパラメータを最適化する様々な方法をご紹介しました。

Google Colaboratoryによる実習はここまでとして、PowerPointの資料での説明に戻りたいと思います。